## importing required Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import string
import re
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
import nltk
from textblob import Word
nltk.download('stopwords')
vocab = Counter()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### stop words in english to remove them from corpus

In [ ]:
stop_words = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer #### wordnet lemmatizer to get the root word 
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

### Helper Functions to remove html tags and square brackets from text

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/imdb_movie_review/IMDB Dataset.csv') ### reading data from google drive

In [ ]:
train.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
train.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


Unique values are less than total count, it means there are few duplicate values in the dataset 

In [ ]:
train.drop_duplicates(inplace = True) ### droping duplicates from dataset

In [ ]:
train['review'][1197]

'WARNING: SPOILER,SPOILER,SPOILER!!!!<br /><br />This is written for filmgoers who may have walked away from "Mood for Love" perplexed and confused about paths the main characters choose in life. From reading other comments and reviews it seems that many viewers and critics missed some very important details which may have prevented them from enjoying this delightful tease of a movie.<br /><br />We are so use to seeing blatant SEX in narrations that we forget that there was a time when filmakers would suggest the "dirty deed" by simply showing the slack-mouthed couples ride off in a sleigh or haywagon only to return into the next scene with a bulging gut or a fat toddler stuck to the hip..."Meet your child".<br /><br />The director chose the same nostalgic approach in telling the story of Mr Chow and Mrs Chan. Last warning...SPOILER SPOILER SPOILER<br /><br />Mr Chow fools Mrs Chan into showing her real emotions when they rehearse his departure forever. Next scene: Mrs Chan leans her h

there are lots of html tags, we need to remove those tags

In [ ]:
train.shape

(49582, 2)

In [ ]:
#Apply function on review column
train['review']=train['review'].apply(denoise_text)
train

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
#function to remove special characters 
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
train['review']=train['review'].apply(remove_special_characters)


In [ ]:


train['review']=train['review'].map(lambda x : x.split())

In [ ]:
train['review'][1197]

['WARNING',
 'SPOILERSPOILERSPOILERThis',
 'is',
 'written',
 'for',
 'filmgoers',
 'who',
 'may',
 'have',
 'walked',
 'away',
 'from',
 'Mood',
 'for',
 'Love',
 'perplexed',
 'and',
 'confused',
 'about',
 'paths',
 'the',
 'main',
 'characters',
 'choose',
 'in',
 'life',
 'From',
 'reading',
 'other',
 'comments',
 'and',
 'reviews',
 'it',
 'seems',
 'that',
 'many',
 'viewers',
 'and',
 'critics',
 'missed',
 'some',
 'very',
 'important',
 'details',
 'which',
 'may',
 'have',
 'prevented',
 'them',
 'from',
 'enjoying',
 'this',
 'delightful',
 'tease',
 'of',
 'a',
 'movieWe',
 'are',
 'so',
 'use',
 'to',
 'seeing',
 'blatant',
 'SEX',
 'in',
 'narrations',
 'that',
 'we',
 'forget',
 'that',
 'there',
 'was',
 'a',
 'time',
 'when',
 'filmakers',
 'would',
 'suggest',
 'the',
 'dirty',
 'deed',
 'by',
 'simply',
 'showing',
 'the',
 'slackmouthed',
 'couples',
 'ride',
 'off',
 'in',
 'a',
 'sleigh',
 'or',
 'haywagon',
 'only',
 'to',
 'return',
 'into',
 'the',
 'next',
 

In [ ]:
re_punc = re.compile('[%s]'%re.escape(string.punctuation))
train['review'] = train['review'].map(lambda x: [word for word in x if word.isalpha()]) ### only alphabets, no no.s and alphanumeric
train['review'] = train['review'].map(lambda x: [w.lower() for w in x]) ### lower case every word
train['review'] = train['review'].map(lambda x:  [w for w in x if not w in stop_words ]) ### stop words removed

In [ ]:
train['review'] = train['review'].map(lambda x: [w for w in x if len(w)>2]) ### words less than 2 letters also dropped

In [ ]:
train

,review,sentiment
0,"[one, reviewers, mentioned, watching, episode,...",positive
1,"[wonderful, little, production, filming, techn...",positive
2,"[thought, wonderful, way, spend, time, hot, su...",positive
3,"[basically, theres, family, little, boy, jake,...",negative
4,"[petter, matteis, love, time, money, visually,...",positive
...,...,...
49995,"[thought, movie, right, good, job, wasnt, crea...",positive
49996,"[bad, plot, bad, dialogue, bad, acting, idioti...",negative
49997,"[catholic, taught, parochial, elementary, scho...",negative
49998,"[going, disagree, previous, comment, side, mal...",negative


In [ ]:

# from spellchecker import SpellChecker
# spell = SpellChecker()
# misspelled = spell.unknown(train['Text'][2])
# print(misspelled)

Not able to find any wrongly spelled word  

In [ ]:
# !pip install pyspellchecker

In [ ]:
train['review'] = train['review'].map(lambda x: [lemmatizer.lemmatize(w) for w in x ]) #### Lemmatization to get root words, does not work as expected

In [ ]:
train['Combined_Text'] = train['review'].map(lambda x: ' '.join([w for w in x])) #### text combined because countvectorizer takes care of preprocessing
train

,review,sentiment,Combined_Text
0,"[one, reviewer, mentioned, watching, episode, ...",positive,one reviewer mentioned watching episode youll ...
1,"[wonderful, little, production, filming, techn...",positive,wonderful little production filming technique ...
2,"[thought, wonderful, way, spend, time, hot, su...",positive,thought wonderful way spend time hot summer we...
3,"[basically, there, family, little, boy, jake, ...",negative,basically there family little boy jake think t...
4,"[petter, matteis, love, time, money, visually,...",positive,petter matteis love time money visually stunni...
...,...,...,...
49995,"[thought, movie, right, good, job, wasnt, crea...",positive,thought movie right good job wasnt creative or...
49996,"[bad, plot, bad, dialogue, bad, acting, idioti...",negative,bad plot bad dialogue bad acting idiotic direc...
49997,"[catholic, taught, parochial, elementary, scho...",negative,catholic taught parochial elementary school nu...
49998,"[going, disagree, previous, comment, side, mal...",negative,going disagree previous comment side maltin on...


In [ ]:
train['sentiment'] = train['sentiment'].map({'positive':1, 'negative':0})
train

,review,sentiment,Combined_Text
0,"[one, reviewer, mentioned, watching, episode, ...",1,one reviewer mentioned watching episode youll ...
1,"[wonderful, little, production, filming, techn...",1,wonderful little production filming technique ...
2,"[thought, wonderful, way, spend, time, hot, su...",1,thought wonderful way spend time hot summer we...
3,"[basically, there, family, little, boy, jake, ...",0,basically there family little boy jake think t...
4,"[petter, matteis, love, time, money, visually,...",1,petter matteis love time money visually stunni...
...,...,...,...
49995,"[thought, movie, right, good, job, wasnt, crea...",1,thought movie right good job wasnt creative or...
49996,"[bad, plot, bad, dialogue, bad, acting, idioti...",0,bad plot bad dialogue bad acting idiotic direc...
49997,"[catholic, taught, parochial, elementary, scho...",0,catholic taught parochial elementary school nu...
49998,"[going, disagree, previous, comment, side, mal...",0,going disagree previous comment side maltin on...


In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(train['Combined_Text'],train['sentiment'],test_size = 0.3,random_state = 24)

In [ ]:
Xtest

1197     warning spoilerspoilerspoilerthis written film...
5808     would rated higher wasnt seriously uneven iris...
6244     possibly stupidest tasteless violent slapstick...
4747     steven buddy remember said try find path least...
35116    marathon interesting premise excellent ambient...
                               ...                        
2851     david cronenberg much like colleague david lyn...
15152    wonder actor acted movie annette bening really...
6292     saw mystery science theater even show couldnt ...
49090    movie start quite gruesomely female bound psyc...
49659    saw beyond rangoon time good first watched saw...
Name: Combined_Text, Length: 14875, dtype: object

### (BOW) Bag of words implementation 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()
# X_train = cv.fit_transform(Xtrain)
# X_test = cv.transform(Xtest).toarray()

In [ ]:
cv=CountVectorizer()#min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
X_train=cv.fit_transform(Xtrain)
#transformed test reviews
X_test=cv.transform(Xtest)

print('BOW_cv_train:',X_train.shape)
print('BOW_cv_test:',X_test.shape)

BOW_cv_train: (34707, 159450)
BOW_cv_test: (14875, 159450)


In [ ]:
cv.vocabulary_

{'hard': 61572,
 'imagine': 68213,
 'director': 36603,
 'capable': 20332,
 'godawful': 57340,
 'crap': 30148,
 'notting': 97149,
 'hill': 64283,
 'pulling': 111306,
 'something': 129695,
 'sensitive': 123969,
 'attractive': 8899,
 'well': 153593,
 'here': 63329,
 'evidence': 45489,
 'quite': 112291,
 'compelling': 27169,
 'several': 124615,
 'alluded': 4035,
 'drama': 39324,
 'yes': 158205,
 'seventy': 124602,
 'play': 106590,
 'today': 142591,
 'feel': 48876,
 'time': 141820,
 'always': 4471,
 'cut': 31758,
 'mainly': 83937,
 'think': 140575,
 'owing': 101995,
 'superlative': 135810,
 'acting': 1050,
 'anne': 5689,
 'reid': 114942,
 'fine': 50525,
 'script': 122302,
 'shadowboxes': 124889,
 'clich': 25245,
 'without': 155854,
 'ever': 45200,
 'getting': 56255,
 'one': 99312,
 'nose': 96791,
 'except': 45787,
 'maybe': 86152,
 'right': 117292,
 'end': 42916,
 'didnt': 35935,
 'like': 80421,
 'either': 41883,
 'tracking': 143898,
 'shot': 126061,
 'indifferent': 69343,
 'goodbye': 57790

Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,ytrain)
pre = lr.predict(X_test)
acc =accuracy_score(ytest,pre)
print(acc*100)

88.25546218487395


In [ ]:
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(X_train,ytrain)
# pre = clf.predict(X_test)
# acc =accuracy_score(ytest,pre)
# print(acc*100)

Random forest classifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,ytrain)
pre = rf.predict(X_test)
acc =accuracy_score(ytest,pre)
print(acc*100)

85.55966386554623


### features limited to 6000 only 

In [ ]:
cv2 = CountVectorizer(max_features=6000) 
X_train = cv2.fit_transform(Xtrain)
X_test = cv2.transform(Xtest)
print(cv2.vocabulary_)

{'hard': 2428, 'imagine': 2655, 'director': 1503, 'capable': 747, 'godawful': 2293, 'crap': 1212, 'hill': 2520, 'pulling': 4165, 'something': 4936, 'sensitive': 4703, 'attractive': 356, 'well': 5840, 'here': 2498, 'evidence': 1844, 'quite': 4205, 'compelling': 1045, 'several': 4733, 'drama': 1598, 'yes': 5981, 'seventy': 4732, 'play': 3953, 'today': 5433, 'feel': 1999, 'time': 5421, 'always': 174, 'cut': 1286, 'mainly': 3252, 'think': 5381, 'acting': 53, 'anne': 218, 'fine': 2047, 'script': 4652, 'clich': 948, 'without': 5913, 'ever': 1835, 'getting': 2258, 'one': 3720, 'nose': 3646, 'except': 1855, 'maybe': 3331, 'right': 4480, 'end': 1749, 'didnt': 1478, 'like': 3125, 'either': 1706, 'tracking': 5483, 'shot': 4795, 'indifferent': 2713, 'goodbye': 2304, 'final': 2041, 'scene': 4624, 'deserved': 1430, 'studied': 5138, 'finale': 2042, 'work': 5934, 'business': 699, 'bit': 529, 'mean': 3336, 'sinking': 4852, 'expectation': 1879, 'mature': 3327, 'man': 3265, 'may': 3330, 'daughter': 1327,

In [ ]:
X_train.shape

(34707, 6000)

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,ytrain)
pre = lr.predict(X_test)
acc =accuracy_score(ytest,pre)
print("Logistic regression after max_features set to 6000 features :",acc*100)

# clf = svm.SVC()
# clf.fit(X_train,ytrain)
# pre = clf.predict(X_test)
# acc =accuracy_score(ytest,pre)
# print("Naive Bayes after max_features set to 6000 features :",acc*100)

rf = RandomForestClassifier()
rf.fit(X_train,ytrain)
pre = rf.predict(X_test)
acc =accuracy_score(ytest,pre)
print("Random forest after max_features set to 6000 features :",acc*100)

Logistic regression after max_features set to 6000 features : 86.56806722689076
Random forest after max_features set to 6000 features : 84.46386554621849


Accuracy of Naive bayes has increased

In [ ]:
cv2.get_feature_names() 

['aaron',
 'abandon',
 'abandoned',
 'abc',
 'ability',
 'able',
 'aboard',
 'abortion',
 'abraham',
 'abrupt',
 'abruptly',
 'absence',
 'absent',
 'absolute',
 'absolutely',
 'absorbing',
 'absurd',
 'absurdity',
 'abuse',
 'abused',
 'abusive',
 'abysmal',
 'academy',
 'accent',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'access',
 'accessible',
 'accident',
 'accidentally',
 'acclaimed',
 'accompanied',
 'accomplish',
 'accomplished',
 'according',
 'account',
 'accuracy',
 'accurate',
 'accurately',
 'accused',
 'ace',
 'achieve',
 'achieved',
 'achievement',
 'achieves',
 'acid',
 'across',
 'act',
 'acted',
 'acting',
 'action',
 'active',
 'activity',
 'actor',
 'actress',
 'actual',
 'actually',
 'adam',
 'adaptation',
 'adapted',
 'adaption',
 'add',
 'added',
 'addict',
 'addicted',
 'addiction',
 'adding',
 'addition',
 'additional',
 'additionally',
 'address',
 'adequate',
 'admirable',
 'admire',
 'admit',
 'admittedly',
 'adolescent

Bi-gram BOW implementation 

In [ ]:
cv3 = CountVectorizer(ngram_range=(2,2)) 
X_train = cv3.fit_transform(Xtrain)
X_test = cv3.transform(Xtest)

lr = LogisticRegression()
lr.fit(X_train,ytrain)
pre = lr.predict(X_test)
acc =accuracy_score(ytest,pre)
print("Logistic regression after max_features set to 6000 features :",acc*100)

# clf = svm.SVC()
# clf.fit(X_train,ytrain)
# pre = clf.predict(X_test)
# acc =accuracy_score(ytest,pre)
# print("Naive Bayes after max_features set to 6000 features :",acc*100)

rf = RandomForestClassifier()
rf.fit(X_train,ytrain)
pre = rf.predict(X_test)
acc =accuracy_score(ytest,pre)
print("Random forest after max_features set to 6000 features :",acc*100)

Logistic regression after max_features set to 6000 features : 85.40504201680672
Random forest after max_features set to 6000 features : 80.65210084033613


TF-Idf implementation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

Xtrain,Xtest,ytrain,ytest = train_test_split(train['Combined_Text'],train['sentiment'],test_size = 0.2,random_state = 24)
vectorizer = TfidfVectorizer(ngram_range = (1,4), max_features = 300000)
X_train = vectorizer.fit_transform(Xtrain)
X_test = vectorizer.transform(Xtest)


In [ ]:
len(vectorizer.get_feature_names())

300000

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,ytrain)
pre = lr.predict(X_test)
acc =accuracy_score(ytest,pre)
print("Logistic regression with TF-IDF from scratch :",acc*100)

# gnb = GaussianNB()
# gnb.fit(X_train,ytrain)
# pre = gnb.predict(X_test)
# acc =accuracy_score(ytest,pre)
# print("Naive Bayes with TF-IDF from scratch :",acc*100)


Logistic regression with TF-IDF from scratch : 89.48270646364827
